参考：https://blog.csdn.net/Monkey_D_Newdun/article/details/79318629

# 1. 爬取歌曲评论数

打开热门歌曲页面，进入到一首歌曲的详细页面，可以看到歌曲的ID，比如《纸短情长（完整版）》的ID是516076896，直接尝试使用BeautifulSoup对链接“http://music.163.com/#/song?id=516076896” 进行解析，发现找不到评论信息。这是因为网易对评论数据采取的是异步传输，所以不能直接通过解析网页内容得到评论数据。打开开发者工具观察,发现奥秘在一个名为“R_SO_4_516076896”的POST请求中。其中R_SO_4_后面的数字516076896为当前歌曲的ID，再观察所上传数据可以看到这里上传了两个参数：params、encSecKey,经过测试发现两个参数对每首歌都是通用的。也就是说将上面链接的歌曲ID替换后传入该参数也能获得数据。再观察服务器所返回的数据可知评论数保存在“total”中（total在“预览”的最后）,其中“comments”存的是评论内容，“hotComments”存的是热门评论内容，如果想扒一扒评论内容就可以从这里获取。注意这些是JSON数据，到时候要解析一下。

In [1]:
from urllib import request  
from bs4 import BeautifulSoup  
import requests  
import json  
import re  
from urllib import parse  
  
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15"}  
  
def comment_count(song_id):  
    url="http://music.163.com/weapi/v1/resource/comments/R_SO_4_"+song_id+"?csrf_token="  
    data = {  
        "params": "j3X2DhLnQqVCgm6VJGe4OT/eQQvjWqPFPmIpAi0lZJGq8hAoELgnjUBBiOqlVx1DvozvOSWr/xkBGosm5lZOIAgioQMQhU77tUibYkOYqJ55sK2c9yBCM5ZkHtBSfAbPb3RBCg4Ps8XNZTGTcw3obZs5l3vQQQLlBVP8jBDTqVBsLLR5gWUzz5x/wnqrLnmq",  
        "encSecKey": "8547739fd6a23087e9b63e003646f65471e9bae3854852f486e28899ba2a556792312d788e9fbf121bceb77c90abf432b38d21f4c157c980616b65c31e9b1f7a8a0eda737296125f5a3d1cb2d660671818636ef7c9543a97cb76f3cddf5d6ebc9d090cda5d60719b1d16d30a6f656a321bd9d491486d6d8e549b7ae2af8b74ba"}  
    #用requests.post()解析这个POST
    req=requests.post(url=url,headers=headers,data=data)  
    req.encoding="utf-8"  
    comment= json.loads(req.text)  
    return comment["total"]  
  
if __name__=="__main__":  
    print(comment_count("516076896"))  

152564


# 2. 爬取热门歌手ID

打开热门歌手页面，歌手信息这里也是采取的异步传输，同样打开Chrome开发者工具观察网页结构，发现奥秘在一个名为“top”的POST请求中。经过测试发现这个POST需要传入完整的Headers（在“标头”中）,不然得不到反馈，但要把“Accept-Encoding:gzip,deflate”去掉，不然到时候解码会报错。这个POST需要传入的数据参数和之前是一样，也是“params”和“encSecKey”两个（也是在“标头”中查看），再观察返回的JSON数据就可以找到歌手的ID和姓名等详细信息了。这次换种方法解析这个POST，不用requests.post()，而用Request+urlopen,虽然麻烦点但是两种方法都熟悉一下。

In [15]:
from urllib import request  
from bs4 import BeautifulSoup  
import requests  
import json  
import re  
from urllib import parse  
  
a_headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15"  
           ,"Referer": "http://music.163.com/discover/artist"  
           ,"Origin": "http://music.163.com"  
           ,"Host":"music.163.com"  
           ,"Cookie":"__utma=94650624.524401346.1510795658.1524743844.1524747113.5; __utmb=94650624.37.10.1524747113; __utmc=94650624; __utmz=94650624.1512478395.2.2.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; JSESSIONID-WYYY=vuk1mFto7DT8F7T6Xks5jZIOI85%2FzTYeHVA%2BWmHD8Zvtiy7FpacVO%5CNpwi%2BVl6U2FRgwUS1bsXgJfU8nrax8zWUxPhBHH8FcfTZjTIxNsY%2FYKivRq22jl65Veqgm0%2BOMiUfmCzxAbMx%2B1UxMqxoUFX1i%2B2kyaGYsfNY%2F1fUw%2FZ32WN7S%3A1524750868394; _iuqxldmzr_=32; WM_TID=Y4lloxguEnAGAGsQVYpDIq4rsdf8OYWU; vinfo_n_f_l_n3=dfacc2c3dca3a054.1.4.1510645015480.1521790673505.1523978281247; vjlast=1510645015.1523978266.12; P_INFO=m13242807107_1@163.com|1523502779|0|urs|00&99|gud&1523163396&mobilemail#gud&440100#10#0#0|&0|mobilemail|m13242807107_1@163.com; _ngd_tid=egOaatuziU0o2o48waZBb5Vs4aFi9Jdp; _ga=GA1.2.463885733.1512733624; __utma=187553192.463885733.1512733624.1514088297.1514088297.1; __utmz=187553192.1514088297.1.1.utmcsr=v.163.com|utmccn=(referral)|utmcmd=referral|utmcct=/special/positivepsychology/; usertrack=ezq0ploqe7Vn9foqB0vDAg==; __gads=ID=e1adaae4fa733fc5:T=1510645013:S=ALNI_Mbq4QIdknEUszE2GBe5Wm7F-nGfXw; _ntes_nnid=70d1039a4031d0bef3e8642d7ac4e310,1510645015721; _ntes_nuid=70d1039a4031d0bef3e8642d7ac4e310; vjuids=789ce4376.15fb975a390.0.b01d3e022c1008; UM_distinctid=15fb9759f315bc-0da7d9adbd6397-6c2c2e4b-fa000-15fb9759f324ab"  
           ,"Content-Type":"application/x-www-form-urlencoded"  
           ,"Content-Length":"412"  
           ,"Connection":"keep-alive"  
           ,"Accept-Language":"zh-cn"  
           ,"Accept":"*/*"}  
  
def artist_id():  
    url="http://music.163.com/weapi/artist/top?csrf_token="  
    data={"params":"q1njkyEPDrJ+FJZzXB7YHkz1+CG29aJAXnlSVERxZjF2pcZkUgeRQD7mYQGB2NEvuirDrX/p+Yc4xhadtCFo/XuXZLsr0a8NlYf2Nsxs0TB1hfWc9zRtEa8smECMxbpN",  
          "encSecKey":"d3df887c50dffc6d09a71abe9fd76d1bc4065e44315609c4feec16fd14707eb2ec67acb75c8093150c95f199832dd38f5528ae805d6b29db8f485435a3a7117dad0cfe6fe327f13c88cbbae9ceebdd9378d375ecbcbdca89fb8f82dd26a5b7281fe1cee68dfd9486636fd217f8eaeb34080131a6acd9586d19aa36386708c9e6"}  
    #这次不用requests.post()，而用Requests和urlopen解析歌手页面的POST，虽然麻烦点但是两种方法都熟悉一下。
    post_data = parse.urlencode(data).encode("utf-8")  
    req=request.Request(url=url,headers=a_headers)  
    res=request.urlopen(req,post_data)  
    html=res.read().decode("utf-8")  
    #解析所返回的JSON数据  
    a_data=json.loads(html)  
    a_list=a_data["artists"]#刚刚观察返回数据可以知道歌手的信息储存在artists键中  
    id_list=[]#用一个列表储存所有歌手的id和name  
    for i in range(len(a_list)):  
        a_dict = {}#用一个字典储存一个歌手的id和name  
        a_dict["name"]=a_list[i]["name"]  
        a_dict["id"]=a_list[i]["id"]  
        id_list.append(a_dict)  
    return id_list  
  
if __name__=="__main__":  
    print(artist_id())  

type(artist_id())
artist_id()

[{'name': '周杰伦', 'id': 6452}, {'name': '陈奕迅', 'id': 2116}, {'name': '薛之谦', 'id': 5781}, {'name': 'BIGBANG', 'id': 126339}, {'name': '林俊杰', 'id': 3684}, {'name': 'Maroon 5', 'id': 96266}, {'name': '王菲', 'id': 9621}, {'name': '李荣浩', 'id': 4292}, {'name': 'G.E.M.邓紫棋', 'id': 7763}, {'name': '张学友', 'id': 6460}, {'name': '杨宗纬', 'id': 6066}, {'name': '许巍', 'id': 5770}, {'name': '蔡健雅', 'id': 7214}, {'name': 'Adele', 'id': 46487}, {'name': 'Bruno Mars', 'id': 178059}, {'name': 'Coldplay', 'id': 89365}, {'name': 'Justin Bieber', 'id': 35531}, {'name': '陈粒', 'id': 1007170}, {'name': '孙燕姿', 'id': 9272}, {'name': '赵雷', 'id': 6731}, {'name': '李志', 'id': 3681}, {'name': '王力宏', 'id': 5346}, {'name': '好妹妹', 'id': 711683}, {'name': 'EXO', 'id': 759509}, {'name': 'Beyond', 'id': 11127}, {'name': 'Alan Walker', 'id': 1045123}, {'name': 'Fall Out Boy', 'id': 56782}, {'name': '金玟岐', 'id': 893259}, {'name': '宋冬野', 'id': 5073}, {'name': '朴树', 'id': 4721}, {'name': 'Eminem', 'id': 32665}, {'name': '李健', 'id': 

[{'id': 6452, 'name': '周杰伦'},
 {'id': 2116, 'name': '陈奕迅'},
 {'id': 5781, 'name': '薛之谦'},
 {'id': 126339, 'name': 'BIGBANG'},
 {'id': 3684, 'name': '林俊杰'},
 {'id': 96266, 'name': 'Maroon 5'},
 {'id': 9621, 'name': '王菲'},
 {'id': 4292, 'name': '李荣浩'},
 {'id': 7763, 'name': 'G.E.M.邓紫棋'},
 {'id': 6460, 'name': '张学友'},
 {'id': 6066, 'name': '杨宗纬'},
 {'id': 5770, 'name': '许巍'},
 {'id': 7214, 'name': '蔡健雅'},
 {'id': 46487, 'name': 'Adele'},
 {'id': 178059, 'name': 'Bruno Mars'},
 {'id': 89365, 'name': 'Coldplay'},
 {'id': 35531, 'name': 'Justin Bieber'},
 {'id': 1007170, 'name': '陈粒'},
 {'id': 9272, 'name': '孙燕姿'},
 {'id': 6731, 'name': '赵雷'},
 {'id': 3681, 'name': '李志'},
 {'id': 5346, 'name': '王力宏'},
 {'id': 711683, 'name': '好妹妹'},
 {'id': 759509, 'name': 'EXO'},
 {'id': 11127, 'name': 'Beyond'},
 {'id': 1045123, 'name': 'Alan Walker'},
 {'id': 56782, 'name': 'Fall Out Boy'},
 {'id': 893259, 'name': '金玟岐'},
 {'id': 5073, 'name': '宋冬野'},
 {'id': 4721, 'name': '朴树'},
 {'id': 32665, 'name': 'E

artist_id是一个函数（上面自己定义的），而artist_id( )是一个list。得到了所有歌手的姓名和ID，并且储存在一个列表中，之后会用到歌手的ID爬取歌曲信息

# 3. 爬取歌手热门单曲ID

首先观察一个歌手的详情页面（如周杰伦），地址栏的id参数就决定歌手页面，我们就爬取热门50单曲的评论信息。这是尝试对页面“http://music.163.com/#/artist?id=6452” 进行解析，发现还是找不到歌曲信息（真是醉了，大公司做项目果然严谨）。打开开发者工具找找玄机，发现有些信息是通过“http://music.163.com/artist?id=6452” （这尼玛少了个#号）（名为artist的POST请求的“标头”里的URL）GET请求发回来的。在返回的数据中还要仔细找一找，可以看到这50首歌曲的信息是储存在class="f-hide"的《ul》标签里。每首歌的信息（ID和歌名）在<a>标签中可以找到。所以用BeautifulSoup对 “http://music.163.com/artist?id=歌手Id” 进行解析，再用find_all("ul",class_="f-hide")就可以找到歌曲信息了

In [25]:
from urllib import request  
from bs4 import BeautifulSoup  
import requests  
import json  
import re  
from urllib import parse
import lxml
  
s_url="http://music.163.com/artist"  #这个网页之所以不是http://music.163.com/artist?id=6452是因为：在get请求中，允许使用params关键字，以一个字典来传递这些参数
#get请求传递url参数，参考：https://blog.csdn.net/u013383813/article/details/76736720
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36"}  
  
def song_id(artist_id):  
    data={"id":artist_id}#将歌手ID作为params参数传入requests.get()方法 
    #python的字典，键必须是不可变对象，比如数，字符串，tuple，frozenset，值可以是任意可以hash化的对象，包括函数，
    #参考：https://zhidao.baidu.com/question/517524357.html
    #在这里data只取到排名第一的周杰伦的数据；在总体代码中可通过设置for循环来遍历所有热门歌手
    req=requests.get(url=s_url,headers=headers,params=data)  
    req.encoding="utf-8"  
    soup=BeautifulSoup(req.text,"lxml")#对返回的数据进行解析  
    song_list=soup.find_all("ul",class_="f-hide")#找到class="f-hide"的<ul>标签  
    song_soup=BeautifulSoup(str(song_list),"lxml")#将<ul>......</ul>再解析一次，以便使用find_all()方法把所有<a>标签取出来  
    song_list=song_soup.find_all("a")  
    id_list=[]#存歌曲ID  
    name_list=[]#存歌名  
    for each in song_list:  
        s_id=each.get("href")#歌曲ID在<a>标签href属性中  
        s_name=each.string  
        s=re.findall(r"\d+",s_id)#用正则找到href中的ID  
        id_list.append(s[0])#由于re.findall()返回的是一个列表，所以用下标将ID取出  
        name_list.append(s_name)  
    return id_list,name_list  
  
if __name__=="__main__":  
    song_id_list,name_list=song_id(6452)  
    print(song_id_list)  
    print(name_list)  

['298317', '210049', '5268423', '536570450', '255020', '210062', '298101', '406346416', '288382', '29808783', '34167344', '288750', '288371', '288820', '462853756', '185667', '185615', '185668', '185616', '185670', '185617', '185672', '185676', '185678', '185680', '185686', '185692', '185694', '185697', '185699', '185701', '185868', '185705', '185613', '185666', '185614', '185618', '185674', '185619', '185620', '185621', '185622', '185644', '185684', '185623', '185696', '185698', '185863', '185703', '185872']
['屋顶', '布拉格广场', '手牵手', '魔术与歌曲：告白气球', '刀马旦', '骑士精神', '祝我生日快乐 (Live)', '海盗', '康定情歌', '夜店咖', '眼泪成诗(Live)', '山歌好比春江水.多谢了(Live)', '黄浦江深', '结尾曲-友谊地久天长(Live)', '可爱小公举', '说了再见', 'Mine Mine', '烟花易冷', '公主病', '好久不见', '你好吗', '雨下一整晚', '我落泪 情绪零碎', '爱的飞行日记', '自导自演', '超人不会飞', '龙战骑士', '给我一首歌的时间', '花海', '说好的幸福呢', '兰亭序', '不能说的秘密', '时光机', '惊叹号', '跨时代', '迷魂曲', '疗伤烧肉粽', '嘻哈空姐', '琴伤', '水手怕水', '世界未末日', '皮影戏', '你是我的OK绷 (Live)', '免费教学录像带', '超跑女神', '蛇舞', '魔术先生', 'Secret (加长快板)', '流浪诗人', '黄金甲']


# 4. 总体代码（热门歌手每人的50首热门歌曲一共的评价数）

这一步主要就是将上面三个部分的代码做一个有机整合就可以了，其实就是用两个循环

In [30]:
from urllib import request  
from bs4 import BeautifulSoup  
import requests  
import json  
import re  
from urllib import parse 


#先爬取热门歌手ID
a_headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/11.1 Safari/605.1.15"  
           ,"Referer": "http://music.163.com/discover/artist"  
           ,"Origin": "http://music.163.com"  
           ,"Host":"music.163.com"  
           ,"Cookie":"__utma=94650624.524401346.1510795658.1524743844.1524747113.5; __utmb=94650624.37.10.1524747113; __utmc=94650624; __utmz=94650624.1512478395.2.2.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; JSESSIONID-WYYY=vuk1mFto7DT8F7T6Xks5jZIOI85%2FzTYeHVA%2BWmHD8Zvtiy7FpacVO%5CNpwi%2BVl6U2FRgwUS1bsXgJfU8nrax8zWUxPhBHH8FcfTZjTIxNsY%2FYKivRq22jl65Veqgm0%2BOMiUfmCzxAbMx%2B1UxMqxoUFX1i%2B2kyaGYsfNY%2F1fUw%2FZ32WN7S%3A1524750868394; _iuqxldmzr_=32; WM_TID=Y4lloxguEnAGAGsQVYpDIq4rsdf8OYWU; vinfo_n_f_l_n3=dfacc2c3dca3a054.1.4.1510645015480.1521790673505.1523978281247; vjlast=1510645015.1523978266.12; P_INFO=m13242807107_1@163.com|1523502779|0|urs|00&99|gud&1523163396&mobilemail#gud&440100#10#0#0|&0|mobilemail|m13242807107_1@163.com; _ngd_tid=egOaatuziU0o2o48waZBb5Vs4aFi9Jdp; _ga=GA1.2.463885733.1512733624; __utma=187553192.463885733.1512733624.1514088297.1514088297.1; __utmz=187553192.1514088297.1.1.utmcsr=v.163.com|utmccn=(referral)|utmcmd=referral|utmcct=/special/positivepsychology/; usertrack=ezq0ploqe7Vn9foqB0vDAg==; __gads=ID=e1adaae4fa733fc5:T=1510645013:S=ALNI_Mbq4QIdknEUszE2GBe5Wm7F-nGfXw; _ntes_nnid=70d1039a4031d0bef3e8642d7ac4e310,1510645015721; _ntes_nuid=70d1039a4031d0bef3e8642d7ac4e310; vjuids=789ce4376.15fb975a390.0.b01d3e022c1008; UM_distinctid=15fb9759f315bc-0da7d9adbd6397-6c2c2e4b-fa000-15fb9759f324ab"  
           ,"Content-Type":"application/x-www-form-urlencoded"  
           ,"Content-Length":"412"  
           ,"Connection":"keep-alive"  
           ,"Accept-Language":"zh-cn"  
           ,"Accept":"*/*"}
s_url="http://music.163.com/artist"  
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36"} 


def artist_id():  
    url="http://music.163.com/weapi/artist/top?csrf_token="  
    data={"params":"q1njkyEPDrJ+FJZzXB7YHkz1+CG29aJAXnlSVERxZjF2pcZkUgeRQD7mYQGB2NEvuirDrX/p+Yc4xhadtCFo/XuXZLsr0a8NlYf2Nsxs0TB1hfWc9zRtEa8smECMxbpN",  
          "encSecKey":"d3df887c50dffc6d09a71abe9fd76d1bc4065e44315609c4feec16fd14707eb2ec67acb75c8093150c95f199832dd38f5528ae805d6b29db8f485435a3a7117dad0cfe6fe327f13c88cbbae9ceebdd9378d375ecbcbdca89fb8f82dd26a5b7281fe1cee68dfd9486636fd217f8eaeb34080131a6acd9586d19aa36386708c9e6"}  
    #这次不用requests.post()，而用Requests和urlopen解析歌手页面的POST，虽然麻烦点但是两种方法都熟悉一下。
    post_data = parse.urlencode(data).encode("utf-8")  
    req=request.Request(url=url,headers=a_headers)  
    res=request.urlopen(req,post_data)  
    html=res.read().decode("utf-8")  
    #解析所返回的JSON数据  
    a_data=json.loads(html)  
    a_list=a_data["artists"]#刚刚观察返回数据可以知道歌手的信息储存在artists键中  
    id_list=[]#用一个列表储存所有歌手的id和name  
    for i in range(len(a_list)):  
        a_dict = {}#用一个字典储存一个歌手的id和name  
        a_dict["name"]=a_list[i]["name"]  
        a_dict["id"]=a_list[i]["id"]  
        id_list.append(a_dict)  
    return id_list 


#再爬取每个歌手的歌曲ID
def song_id(artist_id):  
    data={"id":artist_id}#将歌手ID作为params参数传入requests.get()方法  
    req=requests.get(url=s_url,headers=headers,params=data)  
    req.encoding="utf-8"  
    soup=BeautifulSoup(req.text,"lxml")#对返回的数据进行解析  
    song_list=soup.find_all("ul",class_="f-hide")#找到class="f-hide"的<ul>标签  
    song_soup=BeautifulSoup(str(song_list),"lxml")#将<ul>......</ul>再解析一次，以便使用find_all()方法把所有<a>标签取出来  
    song_list=song_soup.find_all("a")  
    id_list=[]#存歌曲ID  
    name_list=[]#存歌名  
    for each in song_list:  
        s_id=each.get("href")#歌曲ID在<a>标签href属性中  
        s_name=each.string  
        s=re.findall(r"\d+",s_id)#用正则找到href中的ID  
        id_list.append(s[0])#由于re.findall()返回的是一个列表，所以用下标将ID取出  
        name_list.append(s_name)  
    return id_list,name_list 


#最后得到每首歌的评论数量
def comment_count(song_id):  
    url="http://music.163.com/weapi/v1/resource/comments/R_SO_4_"+song_id+"?csrf_token="  
    data = {  
        "params": "j3X2DhLnQqVCgm6VJGe4OT/eQQvjWqPFPmIpAi0lZJGq8hAoELgnjUBBiOqlVx1DvozvOSWr/xkBGosm5lZOIAgioQMQhU77tUibYkOYqJ55sK2c9yBCM5ZkHtBSfAbPb3RBCg4Ps8XNZTGTcw3obZs5l3vQQQLlBVP8jBDTqVBsLLR5gWUzz5x/wnqrLnmq",  
        "encSecKey": "8547739fd6a23087e9b63e003646f65471e9bae3854852f486e28899ba2a556792312d788e9fbf121bceb77c90abf432b38d21f4c157c980616b65c31e9b1f7a8a0eda737296125f5a3d1cb2d660671818636ef7c9543a97cb76f3cddf5d6ebc9d090cda5d60719b1d16d30a6f656a321bd9d491486d6d8e549b7ae2af8b74ba"}  
    #用requests.post()解析这个POST
    req=requests.post(url=url,headers=headers,data=data)  
    req.encoding="utf-8"  
    comment= json.loads(req.text)  
    return comment["total"] 


if __name__=="__main__":  
    f=open("网易云评论.txt","w",encoding="utf-8")  
    id_list=artist_id()  #artist_id()是一个list
    for artist in id_list:  
        total=0  
        f.write("\n\n"+artist["name"]+"热门歌曲以及评论数：")  
        song_id_list,name_list=song_id(artist["id"])  #song_id(artist_id)是先前定义的函数，这里的artist_id由artist["id"]给出（for artist in id_list说明artist是id_list的元素）
        for i in range(len(song_id_list)):  
            if(name_list[i]!=None):#！！！这个地方一定要注意，下面会做说明  
                total+=comment_count(song_id_list[i])  
                f.write(name_list[i]+":"+str(comment_count(song_id_list[i]))+"条")  
        f.write("总计："+str(total)+"条")  
        print(artist["name"]+"网易云音乐热门歌曲总评论数："+str(total)+"条")  
    print("抓取完毕！") 

周杰伦网易云音乐热门歌曲总评论数：712877条
陈奕迅网易云音乐热门歌曲总评论数：1354123条
薛之谦网易云音乐热门歌曲总评论数：3886352条
BIGBANG网易云音乐热门歌曲总评论数：72561条
林俊杰网易云音乐热门歌曲总评论数：1063890条
Maroon 5网易云音乐热门歌曲总评论数：376982条
王菲网易云音乐热门歌曲总评论数：677836条


KeyboardInterrupt: 